In [13]:
'''
第七章:数据清洗
'''
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re  
def ngrams(input, n):
    #这个清洗可以写成函数形式，但这里行数太少，并没有写成函数
    input = re.sub('、', '。', input)
    input = re.sub('，', '。', input)
    input = re.sub('（', '。', input)
    input = re.sub('）', '。', input)
    print(input)
    input = input.split('。')
    output = []
    for i in range(len(input)-n+1):
        output.append(input[i:i+n])
    return output

html = urlopen("https://baike.baidu.com/item/Python/407313")
bsObj = BeautifulSoup(html)
content = bsObj.find("div", {"class":"para"}).get_text()
ngrams = ngrams(content, 2)
print(ngrams)
print("2-grams count is: "+str(len(ngrams)))

Python是一种跨平台的计算机程序设计语言。 是一个高层次的结合了解释性。编译性。互动性和面向对象的脚本语言。最初被设计用于编写自动化脚本(shell)。随着版本的不断更新和语言新功能的添加。越多被用于独立的。大型项目的开发。
[['Python是一种跨平台的计算机程序设计语言', ' 是一个高层次的结合了解释性'], [' 是一个高层次的结合了解释性', '编译性'], ['编译性', '互动性和面向对象的脚本语言'], ['互动性和面向对象的脚本语言', '最初被设计用于编写自动化脚本(shell)'], ['最初被设计用于编写自动化脚本(shell)', '随着版本的不断更新和语言新功能的添加'], ['随着版本的不断更新和语言新功能的添加', '越多被用于独立的'], ['越多被用于独立的', '大型项目的开发'], ['大型项目的开发', '']]
2-grams count is: 8


In [14]:
#OpenRefine软件，可以用作数据筛选以及清洗，类似表格csv
"""
正则表达式两种应用到数据上的方式:
1.value.match(".*([0-9]{4}).*").get(0)
2.re.compile(r'https://blog.csdn.net/.+', re.I).findall(value)
"""

'\n正则表达式两种应用到数据上的方式:\n1.value.match(".*([0-9]{4}).*").get(0)\n2.re.compile(r\'https://blog.csdn.net/.+\', re.I).findall(value)\n'

In [2]:
"""
第八章：自然语言处理
"""
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import string
import operator
def cleanInput(input):
    input = re.sub('\n+', " ", input).lower()
    input = re.sub('\[[0-9]*\]', "", input)
    input = re.sub(' +', " ", input)
    input = bytes(input, "UTF-8")
    input = input.decode("ascii", "ignore")
    cleanInput = []
    input = input.split(' ')
    for item in input:
        item = item.strip(string.punctuation)
        if len(item) > 1 or (item.lower() == 'a' or item.lower() == 'i'):
            cleanInput.append(item)
    return cleanInput

def ngrams(input, n):
    input = cleanInput(input)
    output = {}
    for i in range(len(input)-n+1):
        ngramTemp = " ".join(input[i:i+n])
        if ngramTemp not in output:
            output[ngramTemp] = 0
        output[ngramTemp] += 1
    return output

content = str(urlopen("http://pythonscraping.com/files/inaugurationSpeech.txt").read(),'utf-8')
ngrams = ngrams(content, 2)
sortedNGrams = sorted(ngrams.items(), key = operator.itemgetter(1), reverse=True)
print(sortedNGrams)

[('of the', 213), ('in the', 65), ('to the', 61), ('by the', 41), ('the constitution', 34), ('of our', 29), ('to be', 26), ('the people', 24), ('from the', 24), ('that the', 23), ('and the', 23), ('it is', 23), ('of a', 22), ('may be', 19), ('of their', 19), ('the executive', 19), ('of that', 18), ('for the', 16), ('of its', 16), ('of power', 16), ('have been', 16), ('is the', 16), ('with the', 15), ('the government', 15), ('all the', 15), ('which they', 13), ('has been', 13), ('there is', 13), ('power to', 12), ('the power', 11), ('that of', 11), ('as to', 11), ('their own', 11), ('the same', 11), ('spirit of', 11), ('which the', 10), ('but the', 10), ('which it', 10), ('upon the', 10), ('can be', 10), ('the united', 10), ('united states', 10), ('of all', 10), ('as the', 10), ('part of', 10), ('the character', 10), ('of liberty', 10), ('of my', 9), ('to a', 9), ('it was', 9), ('the great', 9), ('on the', 9), ('of any', 9), ('the president', 9), ('the whole', 9), ('it may', 8), ('of th

In [3]:
from urllib.request import urlopen
from random import randint
def wordListSum(wordList):
    sum = 0
    for word, value in wordList.items():
        sum += value
    return sum

def retrieveRandomWord(wordList):
    randIndex = randint(1, wordListSum(wordList))
    for word, value in wordList.items():
        randIndex -= value
        if randIndex <= 0:
            return word
        
#根据前后单词得出每个单词后面单词的频数，根据这个做后面的类似马尔科夫链的操作 
'''
它的输出类似这个：
{word_a : {word_b : 2, word_c : 1, word_d : 1},
 word_e : {word_b : 5, word_d : 2},...}
'''
def buildWordDict(text):
    # 剔除换行符和引号
    text = text.replace("\n", " ");
    text = text.replace("\"", "");
    # 保证每个标点符号都和前面的单词在一起
    # 这样不会被剔除，保留在马尔可夫链中
    punctuation = [',', '.', ';',':']
    for symbol in punctuation:
        text = text.replace(symbol, " "+symbol+" ");
    words = text.split(" ")
    # 过滤空单词
    words = [word for word in words if word != ""]
    wordDict = {}
    for i in range(1, len(words)):
        if words[i-1] not in wordDict:
        # 为单词新建一个词典
            wordDict[words[i-1]] = {}
        if words[i] not in wordDict[words[i-1]]:
            wordDict[words[i-1]][words[i]] = 0
        wordDict[words[i-1]][words[i]] = wordDict[words[i-1]][words[i]] + 1
    return wordDict

text = str(urlopen("http://pythonscraping.com/files/inaugurationSpeech.txt").read(), 'utf-8')
wordDict = buildWordDict(text)
# 生成链长为100的马尔可夫链
length = 100
chain = ""
currentWord = "I"
for i in range(0, length):
    chain += currentWord+" "
    currentWord = retrieveRandomWord(wordDict[currentWord])
print(chain)

I apprehend , not by citizens are unquestionably to the fabric of one State governments in one individual to increase the public money , we must be that the domestic concerns of high degree of our duty to others of the preservation of the public treasure , from the War of any open warfare may be an argument acquires additional force from the former prosperity . Of the decision there may meet with his services out in their usefulness ends . If the guardianship of conformity to make laws necessary burdens to a country might suppose that the defects of 


In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pymysql
conn = pymysql.connect(host='127.0.0.1',user='root', passwd='123456', db='mysql', charset='utf8')
cur = conn.cursor()
cur.execute("USE csdn")

class SolutionFound(RuntimeError):
    def __init__(self, message):
        self.message = message
        
def getLinks(fromPageId):
    cur.execute("SELECT toPageId FROM links WHERE fromPageId = %s", (fromPageId))
    if cur.rowcount == 0:
        return None
    else:
        return [x[0] for x in cur.fetchall()]
        
def constructDict(currentPageId):
    links = getLinks(currentPageId)
    if links:
        return dict(zip(links, [{}]*len(links)))
    return {}

# 链接树要么为空，要么包含多个链接
def searchDepth(targetPageId, currentPageId, linkTree, depth):
    if depth == 0:
    # 停止递归，返回结果
        return linkTree
    if not linkTree:
        linkTree = constructDict(currentPageId)
        if not linkTree:
        # 若此节点页面无链接，则跳过此节点
            return {}
    if targetPageId in linkTree.keys():
        print("TARGET "+str(targetPageId)+" FOUND!")
        raise SolutionFound("PAGE: "+str(currentPageId))
    for branchKey, branchValue in linkTree.items():
        try:
        # 递归建立链接树
            linkTree[branchKey] = searchDepth(targetPageId, branchKey,branchValue, depth-1)
        except SolutionFound as e:
            print(e.message)
            raise SolutionFound("PAGE: "+str(currentPageId))
    return linkTree
try:
    searchDepth(138, 1, {}, 4)
    print("No solution found")
except SolutionFound as e:
    print(e.message)

TARGET 138 FOUND!
PAGE: 2
PAGE: 1


In [10]:
#NLTK做统计分析和词性分析
# from nltk import FreqDist
# from nltk.book import *
# fdist = FreqDist(text6)
# print( fdist.most_common(10))

In [13]:
# from nltk import word_tokenize, sent_tokenize, pos_tag
# sentences = sent_tokenize("Google is one of the best companies in the world.constantly google myself to see what I'm up to.")
# nouns = ['NN', 'NNS', 'NNP', 'NNPS']
# for sentence in sentences:
#     if "google" in sentence.lower():
#         taggedWords = pos_tag(word_tokenize(sentence))
#         for word in taggleWords:
#             if word[0].lower() == "google" and word[1] in nouns:
#                 print(sentence)
#NTLK的pos_tag对词语的词性判断还是很牛逼的

In [14]:
"""
第九章：穿越网页表单与登录窗口进行采集
"""
import requests
params = {'firstname': 'Ryan', 'lastname': 'Mitchell'}
r = requests.post("http://pythonscraping.com/files/processing.php", data=params)
print(r.text)

Hello there,  !


In [16]:
import requests
params = {'email_addr': 'ryan.e.mitchell@gmail.com'}
r = requests.post("http://post.oreilly.com/client/o/oreilly/forms/quicksignup.cgi", data=params)
print(r.text)

<!DOCTYPE html>
<html lang="en"><head>
  <meta charset="utf-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- The above 3 meta tags *must* come first in the head; any other head content must come *after* these tags -->
  <meta name="description" content="Keep ahead of what's next with the latest ideas, and knowledge you can put into action, in all the subjects you care about, from O'Reilly's community of experts.">
  <meta name="author" content><meta name="robots" content="index, nofollow">

  <link rel="shortcut icon" href="https://www.oreilly.com/favicon.ico" type="image/vnd.microsoft.icon">

  <title>O'Reilly - Subscription Center</title>

  <link rel="stylesheet" type="text/css" href="https://cdn.oreillystatic.com/assets/css/2017_font_face.css">
  <link rel="stylesheet" type="text/css" href="https://cdn.oreillystatic.com/assets/css/subscription-center-topics.css">
  
  <noscript>
    <style>
   

In [19]:
import requests
files = {'uploadFile': open('./m.jpg', 'rb')}
r = requests.post("http://pythonscraping.com/pages/processing2.php", files=files)
print(r.text)

Sorry, there was an error uploading your file.


In [20]:
import requests
params = {'username': 'Ryan', 'password': 'password'}
r = requests.post("http://pythonscraping.com/pages/cookies/welcome.php", params)
print("Cookie is set to:")
print(r.cookies.get_dict())
print("-----------")
print("Going to profile page...")
r = requests.get("http://pythonscraping.com/pages/cookies/profile.php",cookies=r.cookies)
print(r.text)

Cookie is set to:
{'loggedin': '1', 'username': 'Ryan'}
-----------
Going to profile page...
Hey Ryan! Looks like you're still logged into the site!


In [23]:
import requests
session = requests.Session()
params = {'username': 'username', 'password': 'password'}
s = session.post("http://pythonscraping.com/pages/cookies/welcome.php", params)
print("Cookie is set to:")
print(s.cookies.get_dict())
print("-----------")
print("Going to profile page...")
s = session.get("http://pythonscraping.com/pages/cookies/profile.php")
print(s.text)

Cookie is set to:
{'loggedin': '1', 'username': 'username'}
-----------
Going to profile page...
Hey username! Looks like you're still logged into the site!


In [24]:
import requests
from requests.auth import AuthBase
from requests.auth import HTTPBasicAuth
auth = HTTPBasicAuth('ryan', 'password')
r = requests.post(url="http://pythonscraping.com/pages/auth/login.php", auth=
 auth)
print(r.text)

<p>Hello ryan.</p><p>You entered password as your password.</p>


In [25]:
"""
第十章：采集javascript
"""
"""
使用了 Ajax 或 DHTML 技术改变 / 加载内容的页面，可能有一些采集手段，但是用
Python 解决这个问题只有两种途径：直接从 JavaScript 代码里采集内容，或者用 Python 的
第三方库运行 JavaScript，直接采集你在浏览器里看到的页面。
"""

'\n使用了 Ajax 或 DHTML 技术改变 / 加载内容的页面，可能有一些采集手段，但是用\nPython 解决这个问题只有两种途径：直接从 JavaScript 代码里采集内容，或者用 Python 的\n第三方库运行 JavaScript，直接采集你在浏览器里看到的页面。\n'

In [41]:
from selenium import webdriver
import time
driver = webdriver.PhantomJS(executable_path=r'F:\software\phantomjs\phantomjs-2.1.1-windows\bin\phantomjs.exe')
driver.get("http://pythonscraping.com/pages/javascript/ajaxDemo.html")
time.sleep(3)
print(driver.find_element_by_id('content').text)
pageSource = driver.page_source
bsObj = BeautifulSoup(pageSource)
print(bsObj)
driver.close()

Here is some important text you want to retrieve!
A button to click!
<html><head>
<title>Some JavaScript-loaded content</title>
<script src="../js/jquery-2.1.1.min.js"></script>
</head>
<body>
<div id="content">Here is some important text you want to retrieve! <p></p><button id="loadedButton">A button to click!</button></div>
<script>
$.ajax({
    type: "GET",
    url: "loadedContent.php",
    success: function(response){

	setTimeout(function() {
	    $('#content').html(response);
	}, 2000);
    }
  });

function ajax_delay(str){
 setTimeout("str",2000);
}
</script>
</body></html>


In [31]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
html = urlopen("http://pythonscraping.com/pages/javascript/ajaxDemo.html")
bsObj = BeautifulSoup(html.read())
#t = bsObj.findAll()
print(bsObj)
#这里显示的内容之所以和浏览器显示的html的内容大不相同，因为浏览器默认执行script代码，自行生成了新的html内容

<html>
<head>
<title>Some JavaScript-loaded content</title>
<script src="../js/jquery-2.1.1.min.js"></script>
</head>
<body>
<div id="content">
This is some content that will appear on the page while it's loading. You don't care about scraping this.
</div>
<script>
$.ajax({
    type: "GET",
    url: "loadedContent.php",
    success: function(response){

	setTimeout(function() {
	    $('#content').html(response);
	}, 2000);
    }
  });

function ajax_delay(str){
 setTimeout("str",2000);
}
</script>
</body>
</html>


In [43]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.PhantomJS(executable_path=r'F:\software\phantomjs\phantomjs-2.1.1-windows\bin\phantomjs.exe')
driver.get("http://pythonscraping.com/pages/javascript/ajaxDemo.html")
try:
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "loadedButton")))
finally:
    print(driver.find_element_by_id("content").text)
    driver.close()
"""
程序里新导入了一些新的模块，最需要注意的就是 WebDriverWait 和 expected_conditions，
这两个模块组合起来构成了 Selenium 的隐式等待（implicit wait）。
隐式等待与显式等待的不同之处在于，隐式等待是等 DOM 中某个状态发生后再继续运行
代码（没有明确的等待时间，但是有最大等待时限，只要在时限内就可以），而显式等待
明确设置了等待时间，如前面例子的等待三秒钟。
"""
"""
定位器是一种抽象的查询语言，用 By 对象表示，可以用于不同的场合，包括创建选择器。
"""

Here is some important text you want to retrieve!
A button to click!


In [44]:
from selenium import webdriver
import time
from selenium.webdriver.remote.webelement import WebElement
from selenium.common.exceptions import StaleElementReferenceException
def waitForLoad(driver):
    elem = driver.find_element_by_tag_name("html")
    count = 0
    while True:
        count += 1
        if count > 20:
            print("Timing out after 10 seconds and returning")
            return
        time.sleep(.5)
        try:
            elem == driver.find_element_by_tag_name("html")
        except StaleElementReferenceException:
            return
        
driver = webdriver.PhantomJS(executable_path=r'F:\software\phantomjs\phantomjs-2.1.1-windows\bin\phantomjs.exe')
driver.get("http://pythonscraping.com/pages/javascript/redirectDemo1.html")
waitForLoad(driver)
print(driver.page_source)

Timing out after 10 seconds and returning
<html><head>
<title>The Destination Page!</title>

</head>
<body>
This is the page you are looking for!

</body></html>
